# Text-Guided Editing of Images (Using CLIP and StyleGAN)

In [1]:
#@title Setup (may take a few minutes)
!git clone https://github.com/khalilacheche/StyleCLIP.git

import os
os.chdir(f'./StyleCLIP')

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# downloads StyleGAN's weights and facial recognition network weights
ids = ['1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT', '1N0MZSqPRJpLfP4mFQCS14ikrVSe8vQlL']
for file_id in ids:
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.FetchMetadata(fetch_all=True)
  downloaded.GetContentFile(downloaded.metadata['title'])

Cloning into 'StyleCLIP'...
remote: Enumerating objects: 957, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 957 (delta 213), reused 240 (delta 159), pack-reused 618
Receiving objects: 100% (957/957), 256.68 MiB | 31.06 MiB/s, done.
Resolving deltas: 100% (358/358), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-hdrz91vd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-hdrz91vd
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-p

In [24]:
#@title Building Widgets
import ipywidgets as widgets
from IPython.display import display,clear_output

clip_lambda_widget = widgets.FloatText(
    value=1,
    description='CLIP lambda:',
    disabled=False
)
l2_lambda_widget = widgets.FloatText(
    value=0.0004,
    description='L2 lambda:',
    disabled=False
)
loc_lambda_widget = widgets.FloatText(
    value=0.00001,
    description='Loc lambda:',
    disabled=False
)
id_lambda_widget = widgets.FloatText(
    value=0.005,
    description='ID lambda:',
    disabled=False
)
use_stylespace_widget = widgets.Checkbox(
    value=True,
    description='Stylespace',
    disabled=False
)
optimization_steps_widget = widgets.IntText(
    value=40,
    description='Optimization Steps:',
    disabled=False
)

optimization_parameters_widgets = [clip_lambda_widget, l2_lambda_widget,loc_lambda_widget,id_lambda_widget,use_stylespace_widget,optimization_steps_widget] #lambdas, use stylespace, optimization steps
optimization_parameters_box = widgets.VBox(optimization_parameters_widgets)




editing_text_widget = widgets.Text(
    value='A person with black hair',
    placeholder='Type a description of the output image',
    description='Output image desc:',
    disabled=False
)

semantic_parts_options = ["mouth","skin","eyes","nose","ears","eye_brows","hat","hair","neck"]
semantic_parts_widget = widgets.SelectMultiple(
    options=semantic_parts_options,
    value=["hair"],
    #rows=10,
    description='Semantic Parts',
    disabled=False
)
editing_parameters_widgets = [editing_text_widget,semantic_parts_widget] # text, semantic part
editing_parameters_box = widgets.VBox(editing_parameters_widgets)


use_seed_widget = widgets.Checkbox(
    value=True,
    description='Use seed?',
    disabled=False
)

seed_widget = widgets.IntText(
    value=1,
    description='Seed:',
    disabled=False
)

latent_path_widget = widgets.Text(
    value=None,
    placeholder='latens/example.pth',
    description='Latent vector path',
    disabled=False
)
input_image_parameters_widgets = [use_seed_widget,seed_widget,latent_path_widget] # use seed? seed, latent path
input_image_parameters_box = widgets.VBox(input_image_parameters_widgets)




num_scaling_steps_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=9,
    step=2,
    description='Number of images',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


scaling_range_value_widget = widgets.FloatRangeSlider(
    value=[-1, 1],
    min=-2.0,
    max=2.0,
    step=0.001,
    description='Scaling range:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)


create_video_widget = widgets.Checkbox(
    value=False,
    description='Create video?',
    disabled=False
)

export_segmentation_out_widget = widgets.Checkbox(
    value=False,
    description='Export segmentation output?',
    disabled=False
)


output_parameters_widgets = [num_scaling_steps_widget,scaling_range_value_widget,create_video_widget,export_segmentation_out_widget] #create_video? export_segmentation_image? range of scaling, 
output_parameters_box = widgets.VBox(output_parameters_widgets)

accordion = widgets.Accordion(children=[editing_parameters_box,optimization_parameters_box,input_image_parameters_box, output_parameters_box])
accordion.set_title(0, 'Editing Parameters')
accordion.set_title(1, 'Optimization Parameters')
accordion.set_title(2, 'Input Image Parameters')
accordion.set_title(3, 'Output Parameters')

generate_button_widget = widgets.Button(
    description='Generate Image',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to generate',
    icon='check'
)
out = widgets.Output()


In [28]:
#@title Handling Click
import torch
from optimization.run_optimization import main
from argparse import Namespace
from torchvision.utils import make_grid
from torchvision.transforms import ToPILImage

def button_action(b):
  out.clear_output()
  with out:
    print("generate")
    
    args = {
    "description": editing_text_widget.value,
    "ckpt": "stylegan2-ffhq-config-f.pt",
    "stylegan_size": 1024,
    "lr_rampup": 0.05,
    "lr": 0.1,
    "step": optimization_steps_widget.value,
    "clip_lambda": clip_lambda_widget.value,
    "l2_lambda": l2_lambda_widget.value,
    "id_lambda": id_lambda_widget.value,
    "loc_lambda": loc_lambda_widget.value,
    'work_in_stylespace': use_stylespace_widget.value,
    "latent_path": None if use_seed_widget.value else latent_path_widget.value,
    "truncation": 0.7,
    "save_intermediate_image_every": 1 if create_video_widget.value else 20,
    "results_dir": "results",
    "ir_se50_weights": "model_ir_se50.pth",
    "segmentation_model": "face_segmentation",
    "semantic_parts":semantic_parts_widget.value,
    "export_segmentation_image": export_segmentation_out_widget.value
    }
    if (use_seed_widget.value):
      torch.manual_seed(seed_widget.value)
    
    
    
    # run the optimization
    result = main(Namespace(**args))
    result_image = ToPILImage()(make_grid(result.detach().cpu(), normalize=True, scale_each=True, range=(-1, 1), padding=0))
    h, w = result_image.size
    im=result_image.resize((h // 2, w // 2))
    display(im)
    
generate_button_widget.on_click(button_action)

In [30]:
display(accordion,generate_button_widget,out)

Accordion(children=(VBox(children=(Text(value='A person with black hair', description='Output image desc:', pl…

Button(description='Generate Image', icon='check', style=ButtonStyle(), tooltip='Click to generate')

Output()

In [5]:
#@title Create and Download Video

#!ffmpeg -r 15 -i results/%05d.jpg -c:v libx264 -vf fps=25 -pix_fmt yuv420p out.mp4
#from google.colab import files
#files.download('out.mp4')